In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import processing_netcdf as pcdf
import geopandas as gpd
from geopandas import GeoDataFrame
import shapely.geometry 
import numpy as np
from shapely import geometry as gmty
from geofeather import to_geofeather, from_geofeather
import glob
import os
import pyarrow
from xclim import ensembles as ens
from xclim import subset

In [2]:
import matplotlib.pyplot as plt

In [3]:
folder = "/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/"

# Select variable

In [4]:
#Variable 
#var="tg_mean"
#var ="tn_mean"
#var ="tx_mean"
#var ="solidprcptot"
var ="prcptot"
#var ="dlyfrzthw"
#var ="growing_season_length"
#var ="DJC"


files85 = glob.glob(folder+"*"+"rcp85_"+var+"_annual.nc")
files45 = glob.glob(folder+"*"+"rcp45_"+var+"_annual.nc")
filesobs = glob.glob(folder+"NRCAN_obs_"+var+"_annual.nc")

#ex: ACCESS1-3_rcp45_tn_mean_annual.nc
files85
files45


['/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/ACCESS1-3_rcp45_prcptot_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/BNU-ESM_rcp45_prcptot_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CanESM2_rcp45_prcptot_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CMCC-CMS_rcp45_prcptot_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/GFDL-ESM2M_rcp45_prcptot_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/HadGEM2-CC_rcp45_prcptot_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/INM-CM4_rcp45_prcptot_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5A-LR_rcp45_prcptot_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5B-LR_rcp45_prcptot_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/MPI-ESM-LR_rcp45_prcptot_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/NorESM1-M_rcp45_prcptot_annual.nc']

In [5]:
filesobs

['/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/NRCAN_obs_prcptot_annual.nc']

# FOR RCP45 

In [6]:
dsEns45= ens.create_ensemble(files45)
#dsEns45

## Get the percentiles

In [7]:
# Calculate ensembles percentiles on regional means
perctiles45 = ens.ensemble_percentiles(dsEns45)
perctiles45
dsEns45 =0

## Convert in Dataframe

In [8]:
dfper145 = perctiles45.to_dataframe().dropna()
dfper45 = perctiles45.drop('realization').to_dataframe().dropna()
dfmean45 = dfper45.reset_index()
dfper45 = 0

In [9]:
#dfmean45

In [10]:
dfmean45.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcp45"+var+"_forTS.feather")

In [11]:
dfmean45 = pd.read_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcp45"+var+"_forTS.feather")

# FOR RCP85 

In [12]:
dsEns85= ens.create_ensemble(files85)
#dsEns85

## Get the percentiles

In [13]:
# Calculate ensembles percentiles on regional means
perctiles85 = ens.ensemble_percentiles(dsEns85)
perctiles85
dsEns85 =0

## Convert in Dataframe

In [14]:
dfper185 = perctiles85.to_dataframe().dropna()
dfper85 = perctiles85.drop('realization').to_dataframe().dropna()
dfmean85 = dfper85.reset_index()
dfper85 = 0

In [15]:
#dfmean85

In [16]:
dfmean85.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcp85"+var+"_forTS.feather")

In [17]:
dfmean85 = pd.read_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcp85"+var+"_forTS.feather")

# OBSERVED DATA

In [18]:
dsOBs= xr.open_mfdataset(filesobs)

will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  dsOBs= xr.open_mfdataset(filesobs)

to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option to
open_mfdataset).
  combined = auto_combine(



In [19]:
dsOBs

<xarray.Dataset>
Dimensions:  (lat: 510, lon: 1068, time: 64)
Coordinates:
  * time     (time) datetime64[ns] 1950-01-01 1951-01-01 ... 2013-01-01
  * lon      (lon) float32 -140.95833 -140.875 ... -52.125004 -52.04167
  * lat      (lat) float32 83.45833 83.375 83.291664 ... 41.125 41.041668
Data variables:
    prcptot  (time, lat, lon) float32 dask.array<chunksize=(64, 510, 1068), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           NRCAN 10km Gridded Climate Dataset
    history:         2012-10-22T13:14:41: Convert from original format to Net...
    institution:     NRCAN
    source:          ANUSPLIN
    redistribution:  Redistribution policy unknown. For internal use only.

## Convert in Dataframe

In [20]:
dfperObs = dsOBs.to_dataframe().dropna()
dfmeanObs = dfperObs.reset_index()
dfperObs = 0

In [21]:
dfObs = dfmeanObs.rename(columns={var: var+'_Obs'})

In [22]:
dfObs

,lat,lon,time,prcptot_Obs
0,83.125000,-74.541672,1950-01-01,111.560005
1,83.125000,-74.541672,1951-01-01,138.380020
2,83.125000,-74.541672,1952-01-01,105.830017
3,83.125000,-74.541672,1953-01-01,157.129959
4,83.125000,-74.541672,1954-01-01,122.549988
...,...,...,...,...
16464827,41.708332,-82.208336,2009-01-01,792.690430
16464828,41.708332,-82.208336,2010-01-01,793.760071
16464829,41.708332,-82.208336,2011-01-01,1290.789917
16464830,41.708332,-82.208336,2012-01-01,684.479980


In [23]:
dfObs.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcpObs"+var+"_forTS.feather")

In [24]:
dfObs = pd.read_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcpObs"+var+"_forTS.feather")

## Merge 2 rcp dataframes

In [25]:
df2m =  pd.merge(dfmean45, dfmean85, on=["lat","lon", "time"], suffixes=('_45', '_85'))

In [26]:
df2m

,lat,lon,time,prcptot_p10_45,prcptot_p50_45,prcptot_p90_45,prcptot_p10_85,prcptot_p50_85,prcptot_p90_85
0,66.623306,-89.045212,1950-01-01,224.839859,247.529816,308.837433,224.940948,247.548172,308.863007
1,66.623306,-89.045212,1951-01-01,206.217468,216.561401,242.276443,206.774734,219.371414,242.268616
2,66.623306,-89.045212,1952-01-01,178.213516,240.642914,263.426239,178.781281,239.039703,263.340240
3,66.623306,-89.045212,1953-01-01,187.309906,220.098724,290.543091,186.082794,220.423676,290.760651
4,66.623306,-89.045212,1954-01-01,193.584473,221.542145,262.936157,193.157135,222.061340,263.246368
...,...,...,...,...,...,...,...,...,...
11507101,40.041039,-74.045807,2096-01-01,1097.921387,1189.034668,1491.068481,995.721436,1364.554077,1598.562378
11507102,40.041039,-74.045807,2097-01-01,1070.831299,1336.212769,1543.950073,966.680115,1370.195312,1622.534058
11507103,40.041039,-74.045807,2098-01-01,1145.910767,1295.848389,1387.163208,1120.626831,1304.241211,1508.751465
11507104,40.041039,-74.045807,2099-01-01,1175.931030,1287.480103,1407.243530,1070.798584,1361.519897,1436.274048


# Cut by region for rcp45, 85 and Obs

## Get the lat lon regions

In [27]:
regions_folder = "/home/mlopez/EXEC/Grids-polygons-regions/"

In [28]:
files_regions = glob.glob(regions_folder+"*"+"lat_lon.feather")
files_regions

['/home/mlopez/EXEC/Grids-polygons-regions/DDE_STF_20K_REG_FOR_VUE_Slat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/DDE_STF_20K_UA_PER_VUE_Slat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/DOM_BIOlat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/REG_ECOlat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/SDOM_BIOlat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/Secteurs_Operations_Regionaleslat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/SREG_ECOlat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/territoire_guidelat_lon.feather']

In [29]:
listdfreg=[]
for region in files_regions:
    df = pd.read_feather(region)
    listdfreg.append(df)
   

## Create DF by region

In [30]:
short_dict = {"DDE_STF_20K_REG_FOR_VUE_S": ("NM_REG_FOR", "RF"), 
            "DDE_STF_20K_UA_PER_VUE_S": ("PER_NO_UA", "UA"), 
              "DOM_BIO": ("NOM", "DB"), 
              "REG_ECO": ("NOM", "RE"), 
              "SDOM_BIO": ("NOM", "SDB"), 
              "Secteurs_Operations_Regionales": ("D_GENERAL", "SOR"), 
              "SREG_ECO": ("NOM", "SRE"),  
              "territoire_guide": ("TER_GUIDE", "TG") 
              }

for region, (name, short) in short_dict.items():
    print(region, name, short)

DDE_STF_20K_REG_FOR_VUE_S NM_REG_FOR RF
DDE_STF_20K_UA_PER_VUE_S PER_NO_UA UA
DOM_BIO NOM DB
REG_ECO NOM RE
SDOM_BIO NOM SDB
Secteurs_Operations_Regionales D_GENERAL SOR
SREG_ECO NOM SRE
territoire_guide TER_GUIDE TG


In [31]:
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

In [32]:
d = { 'BAS-SAINT-LAURENT':"Bas-Saint-Laurent", 
     'SAGUENAY -LAC-SAINT-JEAN': "Saguenay -Lac-Saint-Jean",
     'CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES':"Capitale-Nationale-Chaudiere-Appalaches",
     'MAURICIE-CENTRE-DU-QUÉBEC':'Mauricie-Centre-du-Quebec','OUTAOUAIS':'Outaouais', 
     'ABITIBI-TEMISCAMINGUE':'Abitibi-Temiscamingue', 'COTE-NORD':'Cote-Nord',
     'NORD-DU-QUEBEC':'Nord-Du-Quebec', 'GASPESIE-ILES-DE-LA-MADELEINE':'Gaspesie-Iles-De-La-Madeleine', 
     'LANAUDIERE':'Lanaudiere','LAURENTIDES':"Laurentides", 
     "ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL":"Estrie-Monteregie-Laval-Montreal", 
     "é": "e", "É": "E", "à": "a", "è": "e", "Î": "i", "È": "E", "ô" : "o", "Ç":"C", "ç":"c",
       } 

## Cutting rcp45 and 85 by region and getting df with 4th degree polynomial

In [33]:
def polysmooth(years, y, degree):
    coef = np.polyfit(years, y, degree)
    return np.polyval(coef,years)

In [34]:
#dfmeanObs = dfdataregObs.groupby(["time", name]).mean() 


In [35]:
#dfmeanObs

In [38]:
for region, (name, short) in short_dict.items():
    print("------------------------------------------------------------------")
    print ("1/7... Cutting by region")
    print(region, name)
    dfregion = pd.read_feather("/home/mlopez/EXEC/Grids-polygons-regions/"+region+"lat_lon.feather")
    print ("2/7... Merging region with data")
    dfdatareg45 = pd.merge(dfregion, df2m, on=["lat","lon"])
    dfdataregObs = pd.merge(dfregion, dfObs, on=["lat","lon"])
    print ("3/7... Get mean by year for every sub-region")
    dfmeanUA45 = dfdatareg45
    dfmean45UA = dfmeanUA45.groupby(["time", name]).mean()
    dfmean45UA2 = dfmean45UA.reset_index()
    dfmeanObs = dfdataregObs.groupby(["time", name]).mean() 
    dfmeanObs2 = dfmeanObs.reset_index()
    print("4/7... Merging 3 cases")
    df3m = pd.merge(dfmean45UA2, dfmeanObs2, how='left', on= ["time", name] )
    print ("5/7... Split by subregion")
    df3ma = df3m[['time', name, var+'_p10_45', var+'_p50_45', var+'_p90_45',
                               var+'_p10_85', var+'_p50_85', var+'_p90_85', var+'_Obs']]
    if (var == "tg_mean" or var== "tn_mean" or var=="tx_mean"):
        df3ma[var+"_Obs"] = df3ma[var+"_Obs"]- 273.15
    if (var == "solidprcptot"):
        df3ma[var+"_Obs"] = df3ma[var+"_Obs"]/10
    df3ma[var+"_Obs"] = round(df3ma[var+"_Obs"], 2)
    for name2,dfsr in df3ma.groupby(name):
        dfsrc = dfsr.copy()
        print ("6/7... 4th Polynomial - rcp45,85")
        years = dfsrc.time.dt.year
        dfsrc["time"] = years#.values
        for p in ('_p10_45', '_p50_45', '_p90_45', '_p10_85', '_p50_85', '_p90_85'):
            if (var == "tg_mean" or var== "tn_mean" or var=="tx_mean"):
                dfsrc[var+p] = polysmooth(years,dfsrc[var+p],4) - 273.15
            if (var == "solidprcptot"):
                dfsrc[var+p] = polysmooth(years,dfsrc[var+p],4)/10
            if (var == "prcptot" or var == "dlyfrzthw" or var =="growing_season_length" or var =="DJC"):
                dfsrc[var+p] = polysmooth(years,dfsrc[var+p],4)
            dfsrc[var+p] = round(dfsrc[var+p], 2) 
        print(" -----------------------7/7 Creating CSV file for sub-region " +name2)
        dfsrc.to_csv(replace_all(name2,d)+"_"+var+"_annual.csv")
    
    
                            
       

------------------------------------------------------------------
1/7... Cutting by region
DDE_STF_20K_REG_FOR_VUE_S NM_REG_FOR
2/7... Merging region with data
3/7... Get mean by year for every sub-region


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3ma[var+"_Obs"] = round(df3ma[var+"_Obs"], 2)



4/7... Merging 3 cases
5/7... Split by subregion
6/7... 4th Polynomial - rcp45,85
 -----------------------7/7 Creating CSV file for sub-region ABITIBI-TEMISCAMINGUE
6/7... 4th Polynomial - rcp45,85
 -----------------------7/7 Creating CSV file for sub-region BAS-SAINT-LAURENT
6/7... 4th Polynomial - rcp45,85
 -----------------------7/7 Creating CSV file for sub-region CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES
6/7... 4th Polynomial - rcp45,85
 -----------------------7/7 Creating CSV file for sub-region COTE-NORD
6/7... 4th Polynomial - rcp45,85
 -----------------------7/7 Creating CSV file for sub-region ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL
6/7... 4th Polynomial - rcp45,85
 -----------------------7/7 Creating CSV file for sub-region GASPESIE-ILES-DE-LA-MADELEINE
6/7... 4th Polynomial - rcp45,85
 -----------------------7/7 Creating CSV file for sub-region LANAUDIERE
6/7... 4th Polynomial - rcp45,85
 -----------------------7/7 Creating CSV file for sub-region LAURENTIDES
6/7... 4th Polynomial -

3/7... Get mean by year for every sub-region
4/7... Merging 3 cases
5/7... Split by subregion
6/7... 4th Polynomial - rcp45,85
 -----------------------7/7 Creating CSV file for sub-region Collines ceinturant le lac Saint-Jean
6/7... 4th Polynomial - rcp45,85
 -----------------------7/7 Creating CSV file for sub-region Collines de Havre-Saint-Pierre et de Blanc-Sablon
6/7... 4th Polynomial - rcp45,85
 -----------------------7/7 Creating CSV file for sub-region Collines de l'Outaouais et du Témiscamingue
6/7... 4th Polynomial - rcp45,85
 -----------------------7/7 Creating CSV file for sub-region Collines de la basse Gatineau
6/7... 4th Polynomial - rcp45,85
 -----------------------7/7 Creating CSV file for sub-region Collines des moyennes Appalaches
6/7... 4th Polynomial - rcp45,85
 -----------------------7/7 Creating CSV file for sub-region Collines du haut Saint-Maurice
6/7... 4th Polynomial - rcp45,85
 -----------------------7/7 Creating CSV file for sub-region Collines du lac Grandm

3/7... Get mean by year for every sub-region
4/7... Merging 3 cases
5/7... Split by subregion
6/7... 4th Polynomial - rcp45,85
 -----------------------7/7 Creating CSV file for sub-region Collines de Saint-Jérôme-Grand-Mère
6/7... 4th Polynomial - rcp45,85
 -----------------------7/7 Creating CSV file for sub-region Collines de la basse Gatineau
6/7... 4th Polynomial - rcp45,85
 -----------------------7/7 Creating CSV file for sub-region Collines de la rivière Vermillon
6/7... 4th Polynomial - rcp45,85
 -----------------------7/7 Creating CSV file for sub-region Collines des lacs Musquanousse et du Vieux Fort
6/7... 4th Polynomial - rcp45,85
 -----------------------7/7 Creating CSV file for sub-region Collines du Grand-Lac-Bostonnais
6/7... 4th Polynomial - rcp45,85
 -----------------------7/7 Creating CSV file for sub-region Collines du Mont-Mégantic
6/7... 4th Polynomial - rcp45,85
 -----------------------7/7 Creating CSV file for sub-region Collines du lac Dumont
6/7... 4th Polynomi

3/7... Get mean by year for every sub-region
4/7... Merging 3 cases
5/7... Split by subregion
6/7... 4th Polynomial - rcp45,85
 -----------------------7/7 Creating CSV file for sub-region 1a
6/7... 4th Polynomial - rcp45,85
 -----------------------7/7 Creating CSV file for sub-region 2a
6/7... 4th Polynomial - rcp45,85
 -----------------------7/7 Creating CSV file for sub-region 2b
6/7... 4th Polynomial - rcp45,85
 -----------------------7/7 Creating CSV file for sub-region 2c
6/7... 4th Polynomial - rcp45,85
 -----------------------7/7 Creating CSV file for sub-region 3ab
6/7... 4th Polynomial - rcp45,85
 -----------------------7/7 Creating CSV file for sub-region 3c
6/7... 4th Polynomial - rcp45,85
 -----------------------7/7 Creating CSV file for sub-region 3d
6/7... 4th Polynomial - rcp45,85
 -----------------------7/7 Creating CSV file for sub-region 4a
6/7... 4th Polynomial - rcp45,85
 -----------------------7/7 Creating CSV file for sub-region 4bc
6/7... 4th Polynomial - rcp45,8